In [1]:
!pip install requests

In [2]:
!pip install spotipy --upgrade

Authentication

In [3]:
import requests
import datetime
import base64 

from urllib.parse import urlencode

In [4]:
client_id = "f8dc0ff4c36b4423b4c145b758d3305f"
client_secret = "dd973168bd04496c83df9bca6382a21b"
user_id = "yjigmwcrybxc77o25bhup38b2"

In [5]:
# do a Look up for a Token
# This is token is for future Requests

In [6]:
client_creds = f"{client_id}:{client_secret}:{user_id}"
type (client_creds)

str

In [7]:
client_creds_b64 = base64.b64encode(client_creds.encode())
type(client_creds_b64)

bytes

In [8]:
base64.b64decode(client_creds_b64)

b'f8dc0ff4c36b4423b4c145b758d3305f:dd973168bd04496c83df9bca6382a21b:yjigmwcrybxc77o25bhup38b2'

In [9]:
token_url = "https://accounts.spotify.com/api/token"
method ="POST"
token_data = {
    "grant_type" : "client_credentials"
}
token_headers = {
    "Authorization" : f"Basic {client_creds_b64.decode()}" # Basic <base64 encoded client_id:client_secret>
}

In [10]:
r = requests.post(token_url, data=token_data, headers=token_headers )
print(r.json())
valid_request = r.status_code in range(200,299)

{'error': 'invalid_client'}


In [11]:
if valid_request:
    token_response_data = r.json()
    now = datetime.datetime.now()
    access_token = token_response_data['access_token']
    expires_in = token_response_data['expires_in'] # seconds
    expires = now + datetime.timedelta(seconds=expires_in)
    did_expire = expires < now

In [12]:
#Base API Client

In [13]:
class SpotifyAPI(object):
    access_token = None
    access_token_expires = datetime.datetime.now()
    access_token_did_expire = True
    client_id = None
    client_secret = None
    token_url = "https://accounts.spotify.com/api/token"
    
    def __init__(self, client_id, client_secret, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.client_id = client_id
        self.client_secret = client_secret

    def get_client_credentials(self):
        """
        Returns a base64 encoded string
        """
        client_id = self.client_id
        client_secret = self.client_secret
        if client_secret == None or client_id == None:
            raise Exception("You must set client_id and client_secret")
        client_creds = f"{client_id}:{client_secret}"
        client_creds_b64 = base64.b64encode(client_creds.encode())
        return client_creds_b64.decode()
    
    def get_token_headers(self):
        client_creds_b64 = self.get_client_credentials()
        return {
            "Authorization": f"Basic {client_creds_b64}"
        }
    
    def get_token_data(self):
        return {
            "grant_type": "client_credentials"
        } 

    
    def perform_auth(self):
        token_url = self.token_url
        token_data = self.get_token_data()
        token_headers = self.get_token_headers()
        r = requests.post(token_url, data=token_data, headers=token_headers)
        if r.status_code not in range(200, 299):
            raise Exception("Could not authenticate client.")
            # return False
        data = r.json()
        now = datetime.datetime.now()
        access_token = data['access_token']
        expires_in = data['expires_in'] # seconds
        expires = now + datetime.timedelta(seconds=expires_in)
        self.access_token = access_token
        self.access_token_expires = expires
        self.access_token_did_expire = expires < now
        return True
    
    def get_access_token(self):
        token = self.access_token
        expires = self.access_token_expires
        now = datetime.datetime.now()
        if expires < now:
            self.perform_auth()
            return self.get_access_token()
        elif token == None:
            self.perform_auth()
            return self.get_access_token() 
        return token
    
    def get_resource_header(self):
        access_token = self.get_access_token()
        headers = {
            "Authorization": f"Bearer {access_token}"
        }
        return headers
        
        
    def get_resource(self, lookup_id, resource_type='playlists', version='v1'):
        endpoint = f"https://api.spotify.com/{version}/{resource_type}/{lookup_id}"
        headers = self.get_resource_header()
        r = requests.get(endpoint, headers=headers)
        if r.status_code not in range(200, 299):
            return {}
        return r.json()
    
    def get_album(self, _id):
        return self.get_resource(_id, resource_type='albums')
    
    def get_artist(self, _id):
        return self.get_resource(_id, resource_type='artists')
    
    
    

    def My_playlists(self, query, type='artist' ): # type
        headers = self.get_resource_header()
        endpoint = "https://api.spotify.com/v1/users/user_id/playlists"
        data = urlencode({"q": query, "type": search_type.lower()})
        lookup_url = f"{endpoint}?{data}"
        r = requests.get(lookup_url, headers=headers)
        if r.status_code not in range(200, 299):  
            return {}
        return r.json()


In [14]:
 spotify = SpotifyAPI(client_id, client_secret)

In [15]:
spotify.perform_auth()

True

In [16]:
access_token = spotify.access_token
access_token

'BQC5GPKyyf-_b6Wb5lNi6VNIXMYlCucvMy_tWdtXcflD_04VNTYLaPP6rMDEOEA-f7FLv5j3wOnkm21tJFs'

In [17]:
#Use Access token

In [18]:
headers = {
     "Authorization" : f"Bearer {access_token}"
}
endpoint = "https://api.spotify.com/v1/search"
data = urlencode({"q" : "Time","type" : "track"})
print(data)

lookup_url = f"{endpoint}?{data}"
print(lookup_url)

r = requests.get(lookup_url, headers=headers)
print(r.status_code)

q=Time&type=track
https://api.spotify.com/v1/search?q=Time&type=track
200


In [19]:
r.json() 

{'tracks': {'href': 'https://api.spotify.com/v1/search?query=Time&type=track&offset=0&limit=20',
  'items': [{'album': {'album_type': 'album',
     'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/0TnOYISbd1XYRBk9myaseg'},
       'href': 'https://api.spotify.com/v1/artists/0TnOYISbd1XYRBk9myaseg',
       'id': '0TnOYISbd1XYRBk9myaseg',
       'name': 'Pitbull',
       'type': 'artist',
       'uri': 'spotify:artist:0TnOYISbd1XYRBk9myaseg'}],
     'available_markets': ['AD',
      'AE',
      'AG',
      'AL',
      'AM',
      'AO',
      'AR',
      'AT',
      'AU',
      'AZ',
      'BA',
      'BB',
      'BD',
      'BE',
      'BF',
      'BG',
      'BH',
      'BI',
      'BJ',
      'BN',
      'BO',
      'BR',
      'BS',
      'BT',
      'BW',
      'BY',
      'BZ',
      'CA',
      'CD',
      'CG',
      'CH',
      'CI',
      'CL',
      'CM',
      'CO',
      'CR',
      'CV',
      'CW',
      'CY',
      'CZ',
      'DE',
      'DJ',
   

In [20]:
#Fetching Data of (The Longest Playlist Ever)

In [21]:
import pandas as pd
import numpy as np
import json

In [22]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

playlist_uri = 'spotify:playlist:2hgBVAx2Q25G0fgmboZzlr'
spotify = spotipy.Spotify(client_credentials_manager=SpotifyClientCredentials(client_id, client_secret))

In [23]:
playlist = spotify.playlist(playlist_uri)
pd.json_normalize(playlist)

,collaborative,description,href,id,images,name,primary_color,public,snapshot_id,type,...,owner.id,owner.type,owner.uri,tracks.href,tracks.items,tracks.limit,tracks.next,tracks.offset,tracks.previous,tracks.total
0,False,literally every song i like (started in nov 20...,https://api.spotify.com/v1/playlists/2hgBVAx2Q...,2hgBVAx2Q25G0fgmboZzlr,"[{'height': None, 'url': 'https://i.scdn.co/im...",the longest playlist i have,None,True,NDYzMSw2N2YyNWFkYTlmN2U0YTQ2MTUzZDRiYzcyYjQ2ZW...,playlist,...,c6z04c5vuymdor6m198v314nz,user,spotify:user:c6z04c5vuymdor6m198v314nz,https://api.spotify.com/v1/playlists/2hgBVAx2Q...,"[{'added_at': '2020-11-16T01:55:10Z', 'added_b...",100,https://api.spotify.com/v1/playlists/2hgBVAx2Q...,0,None,3014


In [24]:
playlist_items = spotify.playlist_items(playlist_uri)
playlist_items

{'href': 'https://api.spotify.com/v1/playlists/2hgBVAx2Q25G0fgmboZzlr/tracks?offset=0&limit=100&additional_types=track%2Cepisode',
 'items': [{'added_at': '2020-11-16T01:55:10Z',
   'added_by': {'external_urls': {'spotify': 'https://open.spotify.com/user/c6z04c5vuymdor6m198v314nz'},
    'href': 'https://api.spotify.com/v1/users/c6z04c5vuymdor6m198v314nz',
    'id': 'c6z04c5vuymdor6m198v314nz',
    'type': 'user',
    'uri': 'spotify:user:c6z04c5vuymdor6m198v314nz'},
   'is_local': False,
   'primary_color': None,
   'track': {'album': {'album_type': 'album',
     'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/0e86yPdC41PGRkLp2Q1Bph'},
       'href': 'https://api.spotify.com/v1/artists/0e86yPdC41PGRkLp2Q1Bph',
       'id': '0e86yPdC41PGRkLp2Q1Bph',
       'name': 'Mother Mother',
       'type': 'artist',
       'uri': 'spotify:artist:0e86yPdC41PGRkLp2Q1Bph'}],
     'available_markets': ['AD',
      'AE',
      'AG',
      'AL',
      'AM',
      'AO',
      '

In [25]:
df = pd.json_normalize(playlist_items['items'])
filter_cols = [col for col in df if col.startswith('track')]
df = df[filter_cols]
df.columns = [col.replace("track.","") for col in df]
df.head()

,album.album_type,album.artists,album.available_markets,album.external_urls.spotify,album.href,album.id,album.images,album.name,album.release_date,album.release_date_precision,...,href,id,is_local,name,popularity,preview_url,track,track_number,type,uri
0,album,[{'external_urls': {'spotify': 'https://open.s...,"[AD, AE, AG, AL, AM, AO, AR, AT, AU, AZ, BA, B...",https://open.spotify.com/album/0X7S0TEZ5NJsSxm...,https://api.spotify.com/v1/albums/0X7S0TEZ5NJs...,0X7S0TEZ5NJsSxmyTnfYi2,"[{'height': 640, 'url': 'https://i.scdn.co/ima...",O My Heart,2008-09-23,day,...,https://api.spotify.com/v1/tracks/2kS6td1yvmpN...,2kS6td1yvmpNgZTt1q5pQq,False,Hayloft,84,https://p.scdn.co/mp3-preview/5b398725e0fe8297...,True,8,track,spotify:track:2kS6td1yvmpNgZTt1q5pQq
1,album,[{'external_urls': {'spotify': 'https://open.s...,"[AD, AE, AG, AL, AM, AO, AR, AT, AU, AZ, BA, B...",https://open.spotify.com/album/40DMb38Z4n5ylfz...,https://api.spotify.com/v1/albums/40DMb38Z4n5y...,40DMb38Z4n5ylfz4n29qt0,"[{'height': 640, 'url': 'https://i.scdn.co/ima...",Touch Up,2007-05-08,day,...,https://api.spotify.com/v1/tracks/266Vb7ItE3o1...,266Vb7ItE3o1q8HdJ9QOvn,False,Verbatim,78,https://p.scdn.co/mp3-preview/fea373245ffd0829...,True,8,track,spotify:track:266Vb7ItE3o1q8HdJ9QOvn
2,album,[{'external_urls': {'spotify': 'https://open.s...,"[AD, AE, AG, AL, AM, AO, AR, AT, AU, AZ, BA, B...",https://open.spotify.com/album/3aqOg25BCgwMQOe...,https://api.spotify.com/v1/albums/3aqOg25BCgwM...,3aqOg25BCgwMQOel1TsE4V,"[{'height': 640, 'url': 'https://i.scdn.co/ima...","Sorry, Mom",2017-02-04,day,...,https://api.spotify.com/v1/tracks/2NBxGmbCTLAr...,2NBxGmbCTLArOiMyWEWsPE,False,I Threw Glass at My Friend's Eyes and Now I'm ...,71,https://p.scdn.co/mp3-preview/05771afb0f74cef7...,True,5,track,spotify:track:2NBxGmbCTLArOiMyWEWsPE
3,album,[{'external_urls': {'spotify': 'https://open.s...,"[CA, US]",https://open.spotify.com/album/6rsQnwaoJHxXJRC...,https://api.spotify.com/v1/albums/6rsQnwaoJHxX...,6rsQnwaoJHxXJRCDBPkBRw,"[{'height': 640, 'url': 'https://i.scdn.co/ima...",Favourite Worst Nightmare (Standard Version),2007-04-24,day,...,https://api.spotify.com/v1/tracks/58ge6dfP91o9...,58ge6dfP91o9oXMzq3XkIS,False,505,87,https://p.scdn.co/mp3-preview/47aca114cd367609...,True,12,track,spotify:track:58ge6dfP91o9oXMzq3XkIS
4,album,[{'external_urls': {'spotify': 'https://open.s...,"[AD, AE, AG, AL, AM, AO, AR, AT, AZ, BA, BB, B...",https://open.spotify.com/album/78bpIziExqiI9qz...,https://api.spotify.com/v1/albums/78bpIziExqiI...,78bpIziExqiI9qztvNFlQu,"[{'height': 640, 'url': 'https://i.scdn.co/ima...",AM,2013-09-09,day,...,https://api.spotify.com/v1/tracks/086myS9r57Ys...,086myS9r57YsLbJpU0TgK9,False,Why'd You Only Call Me When You're High?,90,https://p.scdn.co/mp3-preview/f439aad249c73239...,True,9,track,spotify:track:086myS9r57YsLbJpU0TgK9


In [26]:
# saving the dataframe
df.to_csv(r'C:/Users/twinb/Thesis-Model -Masters/Thesis-Model-Masters-Task 2/Dataset.csv', index=False)

In [27]:
df.columns

Index(['album.album_type', 'album.artists', 'album.available_markets',
       'album.external_urls.spotify', 'album.href', 'album.id', 'album.images',
       'album.name', 'album.release_date', 'album.release_date_precision',
       'album.total_tracks', 'album.type', 'album.uri', 'artists',
       'available_markets', 'disc_number', 'duration_ms', 'episode',
       'explicit', 'external_ids.isrc', 'external_urls.spotify', 'href', 'id',
       'is_local', 'name', 'popularity', 'preview_url', 'track',
       'track_number', 'type', 'uri'],
      dtype='object')

In [28]:
df[['name','album.name']].head(100)

,name,album.name
0,Hayloft,O My Heart
1,Verbatim,Touch Up
2,I Threw Glass at My Friend's Eyes and Now I'm ...,"Sorry, Mom"
3,505,Favourite Worst Nightmare (Standard Version)
4,Why'd You Only Call Me When You're High?,AM
...,...,...
95,I Bet You Look Good On The Dance Floor,Isles Of Wonder: Music For The Opening Ceremon...
96,Rät,Public Void
97,I Bet You Look Good On The Dancefloor,"Whatever People Say I Am, That's What I'm Not"
98,From The Ritz To The Rubble,"Whatever People Say I Am, That's What I'm Not"


In [29]:
df[['album.release_date']].head(10)

,album.release_date
0,2008-09-23
1,2007-05-08
2,2017-02-04
3,2007-04-24
4,2013-09-09
5,2013-04-19
6,2015-10-30
7,2013-09-09
8,2020-07-09
9,2020-07-07


In [30]:
df[['name','track_number']].head(1000)

,name,track_number
0,Hayloft,8
1,Verbatim,8
2,I Threw Glass at My Friend's Eyes and Now I'm ...,5
3,505,12
4,Why'd You Only Call Me When You're High?,9
...,...,...
95,I Bet You Look Good On The Dance Floor,12
96,Rät,7
97,I Bet You Look Good On The Dancefloor,2
98,From The Ritz To The Rubble,12


In [31]:
df['album.artists'].iloc[2]

[{'external_urls': {'spotify': 'https://open.spotify.com/artist/7KeN0XX71T4fGysIYLB5J5'},
  'href': 'https://api.spotify.com/v1/artists/7KeN0XX71T4fGysIYLB5J5',
  'id': '7KeN0XX71T4fGysIYLB5J5',
  'name': 'Destroy Boys',
  'type': 'artist',
  'uri': 'spotify:artist:7KeN0XX71T4fGysIYLB5J5'}]

In [32]:
data = []
for a in df['artists']:
    result = pd.json_normalize(a)
    name = result['name']
    uri = result['uri']
    
    artist_info = spotify.artist(uri[0])
    
    followers = artist_info['followers']['total']
    genres = artist_info['genres']
    popularity = artist_info['popularity']
    
    data.append((name[0], followers, genres, popularity))
    
df1 = pd.DataFrame(data, columns = ('artist', 'followers', 'genres', 'popularity'))

In [33]:
df1['song'] = df['name']
df1['song_popularity'] = df['popularity']

In [34]:
df1['rank'] = range(1, len(df1)+1)
unique_artists = df1[['song','artist', 'followers', 'rank', 'song_popularity','genres']].drop_duplicates(subset=['artist'], keep='first')
unique_artists

,song,artist,followers,rank,song_popularity,genres
0,Hayloft,Mother Mother,2678649,1,84,"[indie pop, vancouver indie]"
2,I Threw Glass at My Friend's Eyes and Now I'm ...,Destroy Boys,252523,3,71,"[indie pop, oakland indie]"
3,505,Arctic Monkeys,14172860,4,87,"[garage rock, permanent wave, rock, sheffield ..."
5,Sweater Weather,The Neighbourhood,8590503,6,95,"[modern alternative rock, modern rock, pop, sh..."
8,Daisy,Ashnikko,1647363,9,76,"[electropop, escape room, pop]"
9,3 Musketeers,NextYoungin,31450,10,71,[viral rap]
10,The Bakery,Melanie Martinez,13298424,11,61,"[alt z, electropop, pop]"
13,Casual Affair,Panic! At The Disco,11263470,14,59,"[modern rock, rock]"
14,bad idea!,girl in red,4764697,15,82,"[bedroom pop, bubblegrunge, indie pop, norwegi..."
15,Teenagers,My Chemical Romance,6728590,16,86,"[emo, pop punk, rock]"


In [35]:
# importing pandas as pd 
import pandas as pd 

df1['rank'] = range(1, len(df1)+1)
unique_artists = df1[['song','artist', 'followers', 'rank', 'song_popularity','genres']].drop_duplicates(subset=['artist'], keep='first')

df1 = pd.DataFrame(unique_artists)

# saving the dataframe
df1.to_csv('dataset2.csv')

In [36]:
# get top  most commom genres
n = 20
top_20=pd.DataFrame(df1['genres'].explode().value_counts()[:n]).reset_index()
top_20.rename(columns = {'index':'Genres','genres':'Total_Count'}, inplace = True)
top_20

,Genres,Total_Count
0,pop,20
1,indie pop,9
2,modern rock,8
3,rock,7
4,dance pop,7
5,hollywood,5
6,post-teen pop,5
7,pop rap,4
8,modern alternative rock,4
9,electropop,4


In [37]:
df1['genres'].explode().value_counts().count()

58

In [52]:
df1.sort_values('song_popularity', ascending = False, inplace = True)
print(df1.head())

                        song             artist  followers  rank  \
5            Sweater Weather  The Neighbourhood    8590503     6   
65       The Real Slim Shady             Eminem   54274838    66   
87            Jocelyn Flores       XXXTENTACION   35670804    88   
24  Still Don't Know My Name           Labrinth    2246276    25   
3                        505     Arctic Monkeys   14172860     4   

    song_popularity                                             genres  
5                95  [modern alternative rock, modern rock, pop, sh...  
65               92                    [detroit hip hop, hip hop, rap]  
87               89                           [emo rap, miami hip hop]  
24               88                             [indie poptimism, pop]  
3                87  [garage rock, permanent wave, rock, sheffield ...  


In [39]:
from collections import Counter
import collections

In [40]:
print(Counter(df1['artist'].values))

Counter({'The Neighbourhood': 1, 'Eminem': 1, 'XXXTENTACION': 1, 'Labrinth': 1, 'Arctic Monkeys': 1, 'MKTO': 1, 'Macklemore & Ryan Lewis': 1, 'My Chemical Romance': 1, 'Cardi B': 1, 'Clairo': 1, 'Of Monsters and Men': 1, 'Lil Peep': 1, 'Mother Mother': 1, 'Beach Bunny': 1, 'girl in red': 1, 'Hozier': 1, 'Billie Eilish': 1, 'The Lumineers': 1, 'Bishop Briggs': 1, 'Penelope Scott': 1, 'John Travolta': 1, 'Sir Chloe': 1, 'Cage The Elephant': 1, 'Ashnikko': 1, 'Ryan McCartan': 1, 'Britney Spears': 1, 'Gwen Stefani': 1, 'Miley Cyrus': 1, 'The Living Tombstone': 1, 'YUNGBLUD': 1, '遠野高志': 1, 'Twenty One Pilots': 1, 'NextYoungin': 1, '100 gecs': 1, 'Christopher Fitzgerald': 1, 'Destroy Boys': 1, 'Amy Winehouse': 1, 'Leslie Odom Jr.': 1, 'Troy': 1, 'ppcocaine': 1, 'AJR': 1, 'Melanie Martinez': 1, 'Panic! At The Disco': 1, 'Nefera DeNile': 1, 'Alli Mauzey': 1, 'MARINA': 1, 'Queen': 1, 'Ricky Montgomery': 1, 'Teddy Hyde': 1})


In [41]:
print(Counter(df1['artist'].values).most_common(5))

[('The Neighbourhood', 1), ('Eminem', 1), ('XXXTENTACION', 1), ('Labrinth', 1), ('Arctic Monkeys', 1)]


In [42]:
#Visualization(Plot them with Echarts)

In [43]:
pip install pyecharts==1.9.1

Note: you may need to restart the kernel to use updated packages.


In [44]:
pip install pandas

Note: you may need to restart the kernel to use updated packages.


In [45]:
pip install echarts-python

In [46]:
import pyecharts
print(pyecharts.__version__)

1.9.1


In [59]:
from pyecharts import options as opts
from pyecharts.charts import Pie

genres = ['Pop','Rock','Indiepop','Modern rock','Garage rock','permanent wave','sheffield indie','electropop','alt z']
data = [40, 25,21 ,18 ,15 ,15,15,10,9,8,8,1]

c = (
    Pie()
    .add(
        "", 
         [
            list(z) 
            for z in zip(
                genres, 
                data,
            )
        ],
          center=["40%", "50%"],
    )
    .set_global_opts(
        title_opts=opts.TitleOpts(title="The Most Used Genres"),
        legend_opts=opts.LegendOpts(type_="scroll", pos_left="80%", orient="vertical"),
    )
    .set_series_opts(label_opts=opts.LabelOpts(formatter="{b}: {c}"))
    
)
c.render_notebook()